In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install torch
!pip install deeppavlov
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

  Using cached deeppavlov-0.17.2-py3-none-any.whl (880 kB)
     |████████████████████████████████| 57 kB 2.7 MB/s 
     |████████████████████████████████| 76 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 2.7 MB/s 
  Using cached Cython-0.29.14-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
     |████████████████████████████████| 53 kB 2.0 MB/s 
  Using cached scikit_learn-0.21.2-cp37-cp37m-manylinux1_x86_64.whl (6.7 MB)
  Using cached uvicorn-0.11.7-py3-none-any.whl (43 kB)
  Using cached prometheus_client-0.7.1.tar.gz (38 kB)
  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Using cached uvloop-0.14.0-cp37-cp37m-manylinux2010_x86_64.whl (3.8 MB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached pytz-2019.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 40 kB 16 kB/s 
  Using cached pandas-0.25.3-cp37-cp37m-manylinux1_x86_64.whl (

In [ ]:
!cp  drive/MyDrive/НИР/dataset_bert.py .
!cp  drive/MyDrive/НИР/functions_for_bert.py .

In [ ]:
from dataset_bert import CustomDataset
from functions_for_bert import Classifier

In [ ]:
from transformers import DistilBertTokenizer
from transformers import DistilBertForSequenceClassification

device = 'cuda'

tokenizer_path = 'DeepPavlov/distilrubert-tiny-cased-conversational'
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_path)

model_path = 'DeepPavlov/distilrubert-tiny-cased-conversational'
model = DistilBertForSequenceClassification.from_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/538 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-tiny-cased-conversational and are newly initialized: ['pr

In [ ]:
import torch

out_features = 768
classifier_DB = Classifier(
    model,
    tokenizer,
    out_features
)

In [ ]:
!nvidia-smi

Fri Jan 14 13:38:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    60W / 149W |   1034MiB / 11441MiB |     30%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path='/content/drive/MyDrive/НИР/',
    train='Join_dataset.csv',
    x = 'text',
    y = 'news'
)

# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid', 'test'],   # fields to which the fiald above will be splitted
    split_proportions=[0.7, 0.2, 0.1],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset
    seed=42)  # seed for iteration over dataset

ImportError: ignored

In [ ]:
x_train, y_train = train_iterator.get_instances(data_type='train')

x_valid, y_valid = train_iterator.get_instances(data_type='valid')

x_test, y_test = train_iterator.get_instances(data_type='test')

print('train: ')
for x, y in list(zip(x_train, y_train))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')

print('validation: ')
for x, y in list(zip(x_valid, y_valid))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')

In [ ]:
classifier_DB.preparation(x_train, y_train, x_valid, y_valid)


In [ ]:
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

all_valid_loss, all_valid_acc, all_train_loss, all_train_acc = classifier_DB.train('DistilRuBERT.pth')

In [ ]:
predictions = [str(classifier_MB.predict(t)) for t in x_test]

In [ ]:
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

In [ ]:
import matplotlib.pyplot as plt 

fig, axs = plt.subplots(2, 2)
axs[0][0].plot([1,2,3,4], all_train_loss)
axs[0][1].plot([1,2,3,4], all_valid_loss)
axs[1][0].plot([1,2,3,4], all_train_acc)
axs[1][1].plot([1,2,3,4], all_valid_acc)

In [ ]:
fig, axs = plt.subplots(2)
axs[0].plot([1,2,3,4], all_train_acc)
axs[1].plot([1,2,3,4], all_valid_acc)